In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
import datetime 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Date set** : movielens data set consist of ratings data for movies

In [2]:
# 100k data
dataset=0
m_cols = ['movieId', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/drive/MyDrive/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),encoding='latin-1')

r_cols = ['userId', 'movieId', 'rating', 'unix_timestamp']
Ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

# data ye MoviesData100k
moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData100k.csv')
moviesData = moviesData.drop('Unnamed: 0', axis=1)
moviesData.head()

,movieId,title,release_date,video_release_date,imdb_url,ratings_no,recom_no
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,452,102
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,131,20
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,90,28
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,209,23
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),86,26


In [3]:
# ## 1m data
# dataset=1
# m_cols = ['movieId','title','Genre']
# movies = pd.read_csv('/content/drive/MyDrive/1m-movieLens/movies.dat', sep='::', names=m_cols, usecols=range(5),encoding='latin-1')

# r_cols = ['userId', 'movieId', 'rating', 'unix_timestamp']
# Ratings = pd.read_csv('/content/drive/MyDrive/1m-movieLens/ratings.dat', sep='::', names=r_cols,encoding='latin-1')

# ## data ye MoviesData1m
# moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData1m.csv')
# moviesData = moviesData.drop('Unnamed: 0', 1)

# moviesData.head()

In [4]:
# ############### create the table of MoviesData 
# ###### add column for no of ratings for each item
# ratings_no =Ratings.groupby('movieId')['rating'].count()
# movies = pd.merge(movies,ratings_no,on='movieId')
# movies.rename(columns = {'rating':'ratings_no'}, inplace = True)

# ###### create a column for no of recommandations each item recieved
# movies['recom_no']=0

# path = '/content/drive/My Drive/MoviesData1m.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   movies.to_csv(f)

# moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData1m.csv')
# moviesData = moviesData.drop('Unnamed: 0', 1)
# moviesData.head()

**test and train data** : split data in β= {0.05, 0.10, 0.15, 0.2}

In [5]:
β= 0.2 #0.05, 0.10, 0.15, 0.2
train, test = train_test_split(Ratings, test_size=β)
# print("\ntrain\n",train.head())
print("Train num.",train.shape)
print("Test num.",test.shape)

Train num. (80000, 4)
Test num. (20000, 4)


In [6]:
train.head()

,userId,movieId,rating,unix_timestamp
9212,148,169,5,877020297
335,184,321,5,889906967
46342,595,844,4,886922069
51283,711,509,4,879993674
31406,479,500,4,879461255


create items vectors with ratings

In [7]:
umRate = pd.pivot_table(train,values='rating',index='userId',columns='movieId')
print("\nUser-Movie dataset:\n")
print("movies no:",len(umRate.columns))
print("users no:",len(umRate.index))
umRate.head() 


User-Movie dataset:

movies no: 1656
users no: 943


movieId,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1674,1675,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,3.0,NaN,4.0,NaN,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#  funtion for compeleting the items in train or test data
def fill_data_table(data):
  # fill movies columns
  x=[]
  for i in data.index:
    x.append(0.0)

  for i in movies['movieId'].values:
    if i not in data.columns:
      z=i.item()  #convert type of numpy.int64 to int for seting loc
      data.insert(z-1, i , x, True)
  
  return data

In [9]:
data = umRate.fillna(0) #fill nan values by 0

# this data may not include all movies , so we should add all of them from movies dataset
data= fill_data_table(data)
data.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.0,4.0,3.0,3.0,0.0,4.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
tumRate = pd.pivot_table(test,values='rating',index='userId',columns='movieId')
tdata = tumRate.fillna(0) #fill nan values by 0

# this data may not include all movies , so we should add all of them from movies dataset 
tdata= fill_data_table(tdata) 
tdata.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Step 1** : hash table function:

calculate hash vector for each item
1. create a random vector v len=n
2. multiply v in each item vector
3. sum the values in each vector
4. create a boolean value: if sum>0 b=1 else b=0
5. repeat the process r times and create a hash table form boolean values


In [11]:
import random 
 
def create_h_table(r):  
  h_table= pd.DataFrame(np.nan, index=[], columns=[])
  # v_table= pd.DataFrame(np.nan, index=[], columns=[])

  for i in range(r):
    # calculate hash for each item
    v0 = [random.randrange(-1,2) for i in range(len(data))] # a random vectore range[-1,1] 
    v_data= data.mul(v0, axis=0) # multiply v to item vectors

    h0= v_data.sum(axis=0)  #h0 = a series of sumation (4)
    # replacing 1 for bigger than 0 and 0 for eagual and smaller than 0 (3)
   
    h0=h0.where(h0<=0,1) 
    h0=h0.where(h0>0,0)
    
    h_table= pd.concat([h_table, h0.to_frame().T])
  
  return h_table

r1=6
h_table=create_h_table(r1)
print("Hash table with r= " , r1)
h_table

Hash table with r=  6


movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


we should have T h_table and set buckets of these tables

so we create a data frame containing all these h_tables and use 

In [12]:
# create T h_table in one dataframe
def create_T_h_table(T,r):
  T_h_table= pd.DataFrame(np.nan, index=[], columns=[])
  for i in range(T):
    h_table=create_h_table(r)
    T_h_table=pd.concat([T_h_table,h_table])
  return T_h_table


# gives i-th h_table
def h_table_num(T_h_table,i,r):
  h_table= T_h_table.iloc[i*r : (i+1)*r]
  return h_table

# T=2
# r=4
# T_H_table= create_T_h_table(T,r)
# T_H_table

In [13]:
# create T h_table in one dataframe
def create_T_buckets(T,r,T_h_table):
  T_buckets= pd.DataFrame(np.nan, index=[], columns=[])
  index=[] # matrix of number of buckets for a table 
  index.append(0)
  for i in range(1,T+1):
    buckets= find_buckets(r,T_h_table.iloc[(i-1)*r : i*r])
    T_buckets=pd.concat([T_buckets,buckets])
    index.append(index[i-1]+len(buckets))
  return T_buckets, index

# gives buckets of i-th h_table
def bucket_num(T_buckets,i, index):
  # buckets of first table i=0
  bucket=T_buckets.iloc[index[i]:index[i+1]]
  return bucket

# T_bucket , index= create_T_buckets(T,r,T_H_table)
# T_bucket

**Step 2:** Establish a set of similar items

create buckets of similar Items with their hash code

In [14]:
# transform hash to bucket name
def bucket_name(r, h_table, movieId):
  c= h_table.columns.get_loc(movieId) 
  bucket_name= h_table.iloc[0:1,c].astype(str)[0][0]
  for row in range(1,r): 
    x= h_table.iloc[row:row+1,c].astype(str)
    bucket_name = bucket_name + x[0][0]
  return bucket_name

# create a dataframe of buckets, containing bucket names (the hash code of bucket movies),
#  size of bucket, and a list of movieId s
def find_buckets(r, h_table):
  buckets= pd.DataFrame(np.nan, index=[], columns=['size', 'movies'])

  for i in h_table.columns:
    # i: movieId
    movie_bucket=bucket_name(r,h_table, i)
    if (buckets.index == movie_bucket).any():
      # bucket is avalible: just add movieId in bucket
      buckets.loc[movie_bucket]['movies'].append(i)
      buckets.at[movie_bucket , 'size'] +=1
    else:
      # new bucket: create a new bucket with size=1 and add  movieId in this bucket
      buckets.loc[movie_bucket]=[ 1, [i]]
  return buckets

bucket=find_buckets(r1,h_table) 
print("buckets of Hash table")
print("number of buckets:", len(bucket))
bucket

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


buckets of Hash table
number of buckets: 64


,size,movies
100101,58.0,"[1, 8, 100, 124, 148, 179, 181, 201, 208, 214,..."
111101,22.0,"[2, 254, 273, 388, 424, 432, 438, 567, 678, 72..."
111000,26.0,"[3, 36, 145, 249, 310, 378, 387, 569, 662, 690..."
100011,23.0,"[4, 44, 95, 364, 481, 592, 703, 713, 764, 828,..."
110011,27.0,"[5, 33, 90, 102, 320, 334, 359, 379, 384, 392,..."
...,...,...
011100,11.0,"[271, 509, 727, 786, 812, 1013, 1029, 1067, 11..."
011001,17.0,"[337, 343, 365, 375, 391, 457, 532, 553, 557, ..."
011110,15.0,"[398, 399, 490, 627, 644, 753, 779, 921, 931, ..."
001101,6.0,"[422, 578, 925, 1035, 1128, 1597]"


**User part**: create an h_table for target user

extract user's history from train data

extract user's similar set from buckets that appears in his_h_table 

In [15]:
# find a list of movieIds which user rated 
def extract_h_set(data, userId):
  # data : train data
  user_table= data[data['userId']==userId]
  h_set= user_table['movieId'] #history set from ratings train data
  return h_set


# create the user h_table from userId and movies he rate
def create_his_h_table(h_table, userId):
  h_set= extract_h_set(train, userId)
  his_h_table= h_table[h_set]
  return his_h_table


# function gives similar set dataframe , that have bucket lists which contains items user rated
def find_sim_set(r,userId, his_h_table, buckets): 
  sim_set= pd.DataFrame(np.nan, index=[], columns=['size', 'rated_no','rated_sum','movies']) 
  # rated_no : number of items user rated in a bucket,
  # rating_sum: sum of ratings (completed in next step)                        

  for i in his_h_table.columns:
    # i: movieId
    movie_bucket= bucket_name(r, his_h_table, i) 
    
    # check if we have bucket in sim_sets or not
    if (sim_set.index == movie_bucket).any():
      sim_set.at[movie_bucket , 'rated_no'] +=1
    else:
      # new bucket: add this bucket which contain movie user rated with rating_no=1 
      sim_set.loc[movie_bucket]=buckets.loc[movie_bucket]
      sim_set.at[movie_bucket , 'size'] =buckets.at[movie_bucket , 'size']
      sim_set.at[movie_bucket , 'rated_no'] =1
  
  # column sum : sum of ratings of rated items in each sim set for a user
  for i in sim_set.index: # for each sim set (i: name of bucket)
    sum=0
    simset= sim_set.loc[i]['movies'] # movies list in bucket 
    for movie in simset: # for each movie in sim set list
      rate= data.loc[userId][movie] 
      sum += rate 
    sim_set.at[i , 'rated_sum'] =sum

  return sim_set

userId=1
user_h_table=create_his_h_table(h_table, userId)
user_sim_set= find_sim_set(6, userId, user_h_table, bucket)
print("similar sets of user id: ",userId,"\nno. of sim sets:", len(user_sim_set))
user_sim_set.head()

similar sets of user id:  1 
no. of sim sets: 55


,size,rated_no,rated_sum,movies
000111,5.0,2.0,7.0,"[192, 211, 500, 579, 848]"
111100,18.0,7.0,33.0,"[47, 48, 66, 81, 175, 224, 270, 581, 1004, 101..."
111111,26.0,3.0,5.0,"[29, 94, 229, 233, 360, 363, 394, 541, 571, 57..."
110001,36.0,7.0,24.0,"[25, 40, 72, 106, 116, 121, 147, 158, 189, 262..."
111000,26.0,3.0,10.0,"[3, 36, 145, 249, 310, 378, 387, 569, 662, 690..."


**Step 3** : Rating Prediction 

1. first find unrated Items for user 
2. calulate rating prediction for each item

In [16]:
# finding unrated items of a user

def xfind_unrated_items(userId): 
  user_table= train[train['userId']==userId] 
  h_set= (user_table['movieId'])-1 #history set from ratings train data
  unrated= movies['movieId'].drop(h_set)
  unrated.sort_values( ascending=True, inplace=True)
  return unrated

def find_unrated_items(userId):
  #  indexes of this series shows itam IDs that are not rated
  user_history= data.loc[userId] 
  unrated= user_history[user_history.values == 0]
  return unrated


userId=1
h_set= extract_h_set(train,userId)
print("user history:\n", h_set)
user_unrated= find_unrated_items(userId)
print("unrated history:\n",user_unrated)
type(user_unrated)

user history:
 26687    211
56312     48
6234     233
32015    147
16455     36
        ... 
74847    116
20952     22
11807    219
78817    216
24896    249
Name: movieId, Length: 207, dtype: int64
unrated history:
 movieId
1       0.0
6       0.0
8       0.0
12      0.0
17      0.0
       ... 
1678    0.0
1679    0.0
1680    0.0
1681    0.0
1682    0.0
Name: 1, Length: 1475, dtype: float64


pandas.core.series.Series

In [17]:
# to use this function we should first create a data frame of columns ['rated_no','rated_sum','predicted_rating'] 

def define_predicting_ratings(userId, iT,sim_set , unrated , predicted_ratings):
  # predicted_ratings= pd.DataFrame(np.zeros, index=[], columns=['rated_no','rated_sum','predicted_rating']) 
  
  for i in sim_set.index: # for each sim set (i: name of bucket)
    simset= sim_set.loc[i]['movies'] # movies list in bucket 
    for movie in simset: # for each movie in sim set list

      if movie in unrated.index and movie in predicted_ratings.index: # movie is unrated and available in pridicted data frame
        predicted_ratings.loc[movie]['rated_no']= predicted_ratings.loc[movie]['rated_no']+ sim_set.at[i,'rated_no']
        predicted_ratings.loc[movie]['rated_sum']= predicted_ratings.loc[movie]['rated_sum']+ sim_set.at[i,'rated_sum']
        predicted_ratings.loc[movie]['predicted_rating']= round((predicted_ratings.loc[movie]['rated_sum']/predicted_ratings.loc[movie]['rated_no']),3)

      elif movie in unrated.index: # movie is unrated but it's not available in pridicted data frame
        x= str(iT)+str(i) #else i
        predicted_ratings.loc[movie]=[x,
                                      sim_set.at[i,'rated_no'] ,
                                      sim_set.at[i,'rated_sum'] , 
                                      round((sim_set.at[i,'rated_sum']/sim_set.at[i,'rated_no']),3) ,
                                      tdata.at[userId,movie]]
  return predicted_ratings
pr= pd.DataFrame(np.zeros, index=[], columns=['index' , 'rated_no','rated_sum','predicted_rating','real_rating']) 
userId=1
pr= define_predicting_ratings(userId,9,user_sim_set , user_unrated, pr)
pr

,index,rated_no,rated_sum,predicted_rating,real_rating
500,9000111,2.0,7.0,3.500,0.0
579,9000111,2.0,7.0,3.500,0.0
848,9000111,2.0,7.0,3.500,0.0
581,9111100,7.0,33.0,4.714,0.0
1004,9111100,7.0,33.0,4.714,0.0
...,...,...,...,...,...
943,9000011,1.0,2.0,2.000,0.0
1175,9000011,1.0,2.0,2.000,0.0
1230,9000011,1.0,2.0,2.000,0.0
1613,9000011,1.0,2.0,2.000,0.0


*Step 4* : Create Recommandation list

1. apply threshold for predicted raitings
2. function to calculate Individual Diversity for a list
3. 

In [18]:
# discount items which ratings are under thershold w 
# sort items in order to ratings 

def apply_thershold_and_sort(w,predicted_ratings):
  # for i in predicted_ratings.index:
  predicted_ratings = predicted_ratings.drop(predicted_ratings.index[predicted_ratings['predicted_rating'] < w])
  predicted_ratings.sort_values(by='predicted_rating', ascending=False, inplace=True)
  return predicted_ratings

pr=apply_thershold_and_sort(4 , pr)
pr.head()

,index,rated_no,rated_sum,predicted_rating,real_rating
1536,9001100,1.0,5.0,5.0,0.0
493,9001110,2.0,10.0,5.0,0.0
503,9001110,2.0,10.0,5.0,0.0
533,9001110,2.0,10.0,5.0,0.0
649,9001110,2.0,10.0,5.0,0.0


In [19]:
from scipy import spatial

def ID (Ru): # Indevidual Diversity
  sum=0
  for i in Ru:
    ratingi=data.loc[:,i]
    for j in Ru:
      if i!=j: 
        ratingj=data.loc[:,j]
        cosine =1 - spatial.distance.cosine(ratingi, ratingj) # max similarity=1 , min similarity=-1
        sum +=cosine
  ID= 1- ( 2/(  len(Ru) * (len(Ru)-1)  ) )* sum
  return ID

Ru=[312, 260, 504, 1016, 189, 735]
ID(Ru)

0.7642198455651357

In [20]:
def create_list(k, predicted_ratings):
  
  mainlist=[]
  pred=[]
  real=[]
  maxID=0
  maxLongtailNo=0
  h= len(predicted_ratings)
  
  for i in range(h):
    items= predicted_ratings[predicted_ratings.real_rating !=0] 
     # list of recoms contain: 0 movieID, 1 no of bucket, 2 pred ratings, 3 real ratings
    newlist=[[],[],[],[]]
    longTailno=0
    
    while len(newlist[0])<k :
      
      # if (choosenitem.rate.test<=0) : put in list
      if len(items)==0:
        break
      x=random.randrange(0,len(items)) # find a random Item
      newItem= items.iloc[x] 
      movieID= items.index.values[x] 
      movieIndex= items.iloc[x]['index']
      movieRealRating= items.iloc[x]['real_rating']
      
      
      if movieIndex not in newlist[1]: #check if it is a repeatetive bucket or not
        newlist[0].append(movieID) # if it wasnt a repeatetive index we put it in the list
        newlist[1].append(movieIndex)
        newlist[2].append(items.iloc[x]['predicted_rating'])
        newlist[3].append(items.iloc[x]['real_rating'])
        if moviesData.loc[moviesData.movieId==movieID,'ratings_no'].item()<longtail_threshold and method != 'main':
          longTailno=longTailno+1

      items=items.drop([movieID]) # then we delete the item from data set whether it was added or not
    
    if len(newlist[0])>1:
      listID= ID(newlist[0])
    else:
      listID=0
    if listID>maxID:
      maxID=listID
      mainlist=newlist[0]
      pred= newlist[2]
      real= newlist[3]
      maxLongtailNo=longTailno

  return mainlist, pred, real, maxID , maxLongtailNo
method='main'
longtail_threshold=80
recomlist, pred, real, maxID , maxLongtailNo= create_list(10,pr)
print(recomlist ,"\n", maxID,"\n", maxLongtailNo)

[46, 148, 245, 268, 104, 119, 154, 123, 202] 
 0.7430860973910018 
 0


In [21]:
def IAcreate_list(k, longtail_threshold ,predicted_ratings,method):
  # k=10
  # longtail_thershold=60
  
  ADlist=[] # max Aggrigate diversity list
  ADlist_ID=0
  ADlist_LongtailNo=0
  maxLongtailNo=0
  maxScore=0
  AD_pred=[]
  AD_real=[]

  IDlist=[] # max individual diversity list
  IDlist_ID=0
  IDlist_LongtailNo=0
  ID_pred=[]
  ID_real=[]
  

  h= len(predicted_ratings)
  for i in range(h): 
    items= predicted_ratings[predicted_ratings.real_rating !=0] ######################################
    newlist=[[],[],[],[]] # list of recoms contain: 0 movieID, 1 no of bucket, 2 pred ratings, 3 real ratings
    longTailno=0
    while len(newlist[0])<k :
      if len(items)==0:
        break
      x=random.randrange(0,len(items)) # find a random Item
      newItem= items.iloc[x] 
      movieID= items.index.values[x] 
      movieIndex= items.iloc[x]['index']
      movieRealRating= items.iloc[x]['real_rating']
      
      
      # if movieRealRating!=0:
      if movieIndex not in newlist[1]: #check if it is a repeatetive bucket or not
        newlist[0].append(movieID) # if it wasnt a repeatetive index we put it in the list
        newlist[1].append(movieIndex)
        newlist[2].append(items.iloc[x]['predicted_rating'])
        newlist[3].append(items.iloc[x]['real_rating'])
        if moviesData.loc[moviesData.movieId==movieID,'ratings_no'].item()<longtail_threshold:
          longTailno=longTailno+1
        
      items=items.drop([movieID]) # then we delete the item from data set whether it was added or not
    
    #score of ID
    if len(newlist[0])>1:
      listID= ID(newlist[0])
      longTailScore= longTailno/len(newlist[0]) 
    else:
      listID=0
      longTailScore= longTailno
    
    if listID > IDlist_ID:      
      IDlist=newlist[0]
      ID_pred= newlist[2]
      ID_real= newlist[3]
      IDlist_ID=listID
      IDlist_LongtailNo=longTailno
    
    # score of longtail items
    # longTailScore= longTailno/len(newlist[0]) 
    # score of list , combination of ID and number of longtail items, max=2 min=0
    
    a=0.5
    listScore= a*listID + (1-a)*longTailScore
    
    if method=='max longtail':

      if longTailno > maxLongtailNo:
        maxLongtailNo=longTailno
        ADlist=newlist[0]
        AD_pred= newlist[2]
        AD_real= newlist[3]
        ADlist_ID=listID
        ADlist_LongtailNo=longTailno
        
      elif maxLongtailNo==0: 
        maxLongtailNo=longTailno
        ADlist=IDlist
        AD_pred= ID_pred
        AD_real= ID_real
        ADlist_ID=IDlist_ID
        ADlist_LongtailNo=IDlist_LongtailNo
    
    elif method=='longtail score':
      if listScore > maxScore:
        maxScore=listScore
        ADlist_ID=listID
        ADlist=newlist[0]
        AD_pred= newlist[2]
        AD_real= newlist[3]
        ADlist_LongtailNo=longTailno

  return IDlist, ID_pred, ID_real, IDlist_ID, IDlist_LongtailNo, ADlist, AD_pred, AD_real, ADlist_ID , ADlist_LongtailNo

IDlist, ID_pred, ID_real, IDlist_ID, IDlist_LongtailNo, ADlist, AD_pred, AD_real, ADlist_ID , ADlist_LongtailNo= IAcreate_list(10,60,pr,'longtail score')
print(IDlist,"\n", IDlist_ID ,"\n", IDlist_LongtailNo,"\n", ADlist, "\n", ADlist_ID ,"\n", ADlist_LongtailNo)

[46, 202, 148, 245, 119, 62, 268, 104, 154] 
 0.751338602153004 
 3 
 [46, 202, 148, 245, 119, 62, 268, 104, 154] 
 0.751338602153004 
 3


In [22]:
def create_ADlist(k, longtail_threshold, predicted_ratings):

  mainlist=[]
  pred=[]
  real=[]
  maxScore=0
  maxID=0
  maxLongtailNo=0
  h= len(predicted_ratings)
  
  for i in range(h):  
    items= predicted_ratings[predicted_ratings.real_rating !=0] 
    newlist=[[],[],[],[]]
    longTailno=0
    while len(newlist[0])<k :

      if len(items)==0:
        break
      x=random.randrange(0,len(items)) # find a random Item
      newItem= items.iloc[x] 
      movieID= items.index.values[x] 
      movieIndex= items.iloc[x]['index']
      movieRealRating= items.iloc[x]['real_rating']
      
      # if movieRealRating!=0:
      if movieIndex not in newlist[1]: #check if it is a repeatetive bucket or not
        newlist[0].append(movieID) # if it wasnt a repeatetive index we put it in the list
        newlist[1].append(movieIndex)
        newlist[2].append(items.iloc[x]['predicted_rating'])
        newlist[3].append(items.iloc[x]['real_rating'])
        if moviesData.loc[moviesData.movieId==movieID,'ratings_no'].item()<longtail_threshold:
          longTailno=longTailno+1
      
      items=items.drop([movieID]) # then we delete the item from data set whether it was added or not
    
    # score of longtail items
    if len(newlist[0]) >0: longTailScore= longTailno/len(newlist[0]) 
    else : longTailScore=0
    
    # score of ID
    if len(newlist[0])>1: 
      listID= ID(newlist[0])
    else:
      listID=0
    # score of list , combination of ID and number of longtail items, max=2 min=0
    a=0.5
    listScore= a*listID + (1-a)*longTailScore

    if listScore > maxScore:
      mainlist=newlist[0]
      maxScore=listScore
      maxID=listID
      maxLongtailNo=longTailno
      pred= newlist[2]
      real= newlist[3]

  return mainlist, pred, real ,maxScore , maxID, maxLongtailNo

recomlist, pred, real,maxScore, maxID, maxLongtailNo= create_ADlist(10,60,pr)
print(recomlist,"\n", maxScore, "\n", maxID, "\n", maxLongtailNo) 

[190, 104, 123, 46, 154, 245, 148, 119, 202] 
 0.5390217630914484 
 0.7447101928495636 
 3


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

y_actual = [1,2,3,4,5]
y_predicted = [1.6,2.5,2.9,3,4.1]

def Evaluation(pred, real):
  MAE= mean_absolute_error(real, pred)
  MSE = mean_squared_error(real, pred) 
  RMSE = math.sqrt(MSE)
  return MAE , RMSE

# movies['bool_recom']=0
def aggregate_diversity(recommendations):
  movies['bool_recom']=0
  for user_list in recommendations.index:
    for item in recommendations.loc[user_list,'recom']:
      movies.loc[movies.movieId==item,'bool_recom']=1
  AD= movies['bool_recom'].sum()
  return AD

In [24]:
# functions for counting the number of times each item is recommended

def update_MoviesData(dataset, recommendations):
  # if dataset==0 :
  #   moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData100k.csv')
  # else:
  #   moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData1m.csv')
  # moviesData = moviesData.drop('Unnamed: 0', axis=1)

  for user_list in recommendations.index:
    for item in recommendations.loc[user_list,'recom']:
      a = moviesData.loc[moviesData.movieId==item,'recom_no']
      moviesData.loc[moviesData.movieId==item,'recom_no']= a+1

  if dataset==0 :
    path = '/content/drive/My Drive/MoviesData100k.csv'  #****** 100 k *******
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      moviesData.to_csv(f)
  else:
    path = '/content/drive/My Drive/MoviesData1m.csv'  #****** 1 m  *******
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      moviesData.to_csv(f)

def update_MoviesData_perUser(dataset, recomlist):
  for item in recomlist:
      a = moviesData.loc[moviesData.movieId==item,'recom_no']
      moviesData.loc[moviesData.movieId==item,'recom_no']= a+1  
  
  if dataset==0 :
    path = '/content/drive/My Drive/MoviesData100k.csv'  #****** 100 k *******
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      moviesData.to_csv(f)
  else:
    path = '/content/drive/My Drive/MoviesData1m.csv'  #****** 1 m  *******
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      moviesData.to_csv(f)


**Main**

In [25]:
global r
r=6 #4,6,8,10   number of hash code for each item
T=4 #4,6,8,10   number of hash table
w=3.5 # predicted rating thershold

T_H_table= create_T_h_table(T,r)
T_bucket , index = create_T_buckets(T,r,T_H_table)

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


create list for one user

In [26]:
# userId=1
# user_predicted_ratings= pd.DataFrame(np.zeros, index=[], columns=['index','rated_no','rated_sum','predicted_rating','real_rating']) 
# user_unrated= find_unrated_items(userId)
# for i in range(T):
#   h_table= h_table_num(T_H_table,i,r)
#   bucket= bucket_num(T_bucket,i, index)
#   user_h_table= create_his_h_table(h_table ,userId) 
#   user_sim_set= find_sim_set(r, userId, user_h_table, bucket) 
#   user_predicted_ratings= define_predicting_ratings(userId, i, user_sim_set, user_unrated, user_predicted_ratings) 
#   user_predicted_ratings= apply_thershold_and_sort(w,user_predicted_ratings)     # i_set

# user_recom , user_pred, user_real, user_diversity= create_list(k ,user_predicted_ratings) 
# user_recom 

the main method without improvement: DiveRec_LSH

In [27]:
import datetime 
recommendations= pd.DataFrame(np.zeros, index=[], columns=['userId', 'ID', 'MAE', 'RMSE', 'recom', 'pred', 'real', 'time']) 
u=1 #number of users for giving recommandations for experiment
k=5   # number of item recommmandation in a list 
method= 'main'
while (len(recommendations) <u):
  rand = random.randrange(1,len(tdata.index))
  userId= tdata.iloc[rand].name

  user_predicted_ratings= pd.DataFrame(np.zeros, index=[], columns=['index','rated_no','rated_sum','predicted_rating','real_rating']) 
  user_unrated= find_unrated_items(userId)

  for i in range(T):
    h_table= h_table_num(T_H_table,i,r) 
    bucket= bucket_num(T_bucket,i, index) 
    user_h_table= create_his_h_table(h_table ,userId) 
    user_sim_set= find_sim_set(r, userId, user_h_table, bucket) 
    user_predicted_ratings= define_predicting_ratings(userId, i, user_sim_set, user_unrated, user_predicted_ratings) 
    user_predicted_ratings= apply_thershold_and_sort(w,user_predicted_ratings)     # i_set

  user_recom , user_pred, user_real, user_diversity, longtailno= create_list(k ,user_predicted_ratings) 
  
  #calculate RMSE , MAE for user
  if len(user_pred)!=0:
    MAE,RMSE= Evaluation(user_pred, user_real)
  
    recommendations.loc[userId]=[userId, user_diversity, MAE, RMSE, user_recom, user_pred, user_real, datetime.datetime.now()]
    print(recommendations.loc[userId], user_recom) 


# update MoviesData
update_MoviesData(dataset, recommendations)

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


userId                            689
ID                           0.571396
MAE                            0.5334
RMSE                         0.699269
recom       [109, 125, 596, 475, 328]
pred      [4.0, 3.667, 4.0, 4.0, 5.0]
real        [5.0, 3.0, 3.0, 4.0, 5.0]
time       2023-02-27 22:50:53.024485
Name: 689, dtype: object [109, 125, 596, 475, 328]


In [28]:
def reRanking(k, recomlist, pred, real, method):
  # reclist=[1,2,3,4,5,7,8]
  # moviesData.loc[moviesData.movieId==item,'recom_no']= a+1
  dataRec=pd.DataFrame(np.zeros, index=[], columns=[]) 
  for item in recomlist:
    dataRec=dataRec.append(moviesData.loc[moviesData['movieId'] == item])

  # dataRec= dataRec[['movieId','ratings_no','recom_no']]
  dataRec['pred']=pred
  dataRec['real']=real
  
  #  sort by recom and rating priority
  if method=='reRanking-recom priority':
    dataRec.sort_values(by=['recom_no','ratings_no'], ascending=True ,inplace=True)
  elif method=='reRanking-ratings priority':
    dataRec.sort_values(by=['ratings_no','recom_no'], ascending=True ,inplace=True)
  elif method=='reRanking-score log2':
    # score sort by (log ratig + recom )
    dataRec['score']= np.log2(dataRec['ratings_no']) + dataRec['recom_no']   ### log 2
    dataRec.sort_values(by=['score'], ascending=True ,inplace=True)
  elif method=='reRanking-score log 1.5':
    base=1.5
    dataRec['score']= (np.log(dataRec['ratings_no']) / np.log(base)) + dataRec['recom_no']
    dataRec.sort_values(by=['score'], ascending=True ,inplace=True)
  
  longtailno=0
  if len(recomlist)>=k:
    for i in range(k):
      if dataRec.iloc[i]['ratings_no']<= longtail_threshold:
        longtailno=longtailno+1

  recomlist=dataRec['movieId'].tolist()
  pred=dataRec['pred'].tolist()
  real=dataRec['real'].tolist()
  print(dataRec)
  return recomlist[:k], pred[:k], real[:k],longtailno


In [29]:
# experiments for reranking

k=5   # number of item recommmandation in a list 
N=15  # number of items before reranking, set it to {8,10,15}
u=1  # =20 for experiments number of users for giving recommandations for experiment
longtail_threshold=80 # {60,80,100,200 for 100k dataset / 270,400,500,750 for 1M dataset}

IDrecommendations= pd.DataFrame(np.zeros, index=[], columns=['userId', 'ID','longtail no.', 'MAE', 'RMSE', 'recom', 'pred', 'real', 'time']) 
ADrecommendations= pd.DataFrame(np.zeros, index=[], columns=['userId', 'ID','longtail no.', 'MAE', 'RMSE', 'recom', 'pred', 'real', 'time']) 
method = 'reRanking-recom priority' #  'reRanking-recom priority' 'reRanking-ratings priority'  'reRanking-score log2'   'reRanking-score log 1.5'

ADtime=0
IDtime=0


while (len(IDrecommendations) < u):
  rand = random.randrange(1,len(tdata.index))
  userId= tdata.iloc[rand].name
  
  user_predicted_ratings= pd.DataFrame(np.zeros, index=[], columns=['index','rated_no','rated_sum','predicted_rating','real_rating']) 
  user_unrated= find_unrated_items(userId)

  for i in range(T):
    h_table= h_table_num(T_H_table,i,r) 
    bucket= bucket_num(T_bucket,i, index) 
    user_h_table= create_his_h_table(h_table ,userId) 
    user_sim_set= find_sim_set(r, userId, user_h_table, bucket) 
    user_predicted_ratings= define_predicting_ratings(userId, i, user_sim_set, user_unrated, user_predicted_ratings) 
    user_predicted_ratings= apply_thershold_and_sort(w,user_predicted_ratings)     # i_set



  # Create ID rec list
  start_time1= datetime.datetime.now()
  user_recom , user_pred, user_real, user_diversity, longtailno= create_list(k ,user_predicted_ratings) 
  end_time1= datetime.datetime.now()
  delta1 = end_time1 - start_time1
  IDtime= delta1.seconds + IDtime
  print(u-len(IDrecommendations),'ID',user_recom,IDtime)

  
  if len(user_recom)>1:
    MAE,RMSE= Evaluation(user_pred, user_real) #calculate RMSE , MAE for user
    IDrecommendations.loc[userId]=[userId, user_diversity, longtailno, MAE, RMSE, user_recom, user_pred, user_real, datetime.datetime.now()]
    # update_MoviesData_perUser(dataset, user_recom) 

  # Create AD rec list
    start_time2= datetime.datetime.now()
    user_recom , user_pred, user_real, user_diversity, longtailno= create_list(N ,user_predicted_ratings) 
    user_recom , user_pred, user_real, longtailno= reRanking(k, user_recom , user_pred, user_real, method)
    end_time2= datetime.datetime.now()
    delta2 = end_time2 - start_time2
    ADtime= delta2.seconds + ADtime

  # score of ID
    user_diversity= ID(user_recom)
  
  # calculate RMSE , MAE for user
    MAE,RMSE= Evaluation(user_pred, user_real)
    
    ADrecommendations.loc[userId]=[userId, user_diversity, longtailno, MAE, RMSE, user_recom, user_pred, user_real, datetime.datetime.now()]
    print(userId,'AD',user_recom)
    update_MoviesData_perUser(dataset, user_recom) 
    

# update MoviesData
if dataset==0 :
  moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData100k.csv')
else:
  moviesData = pd.read_csv('/content/drive/MyDrive/MoviesData1m.csv')
moviesData = moviesData.drop('Unnamed: 0', axis=1)
update_MoviesData(dataset, IDrecommendations)
update_MoviesData(dataset, ADrecommendations) 

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


1 ID [115, 10, 165, 343, 535] 24
     movieId                                              title release_date  \
114      115    Haunted World of Edward D. Wood Jr., The (1995)  26-Apr-1996   
660      661                                   High Noon (1952)  01-Jan-1952   
527      528                         Killing Fields, The (1984)  01-Jan-1984   
588      589                             Wild Bunch, The (1969)  01-Jan-1969   
534      535                            Addicted to Love (1997)  23-May-1997   
143      144                                    Die Hard (1988)  01-Jan-1988   
656      657                   Manchurian Candidate, The (1962)  01-Jan-1962   
248      249  Austin Powers: International Man of Mystery (1...  02-May-1997   
9         10                                 Richard III (1995)  22-Jan-1996   
281      282                             Time to Kill, A (1996)  13-Jul-1996   
342      343                         Alien: Resurrection (1997)  01-Jan-1997   
297    

In [38]:
# for choosing the best list
import datetime 
k=5   # number of item recommmandation in a list 
u=1  # =20 or 50 for experiments, number of users for giving recommandations for experiment
longtail_threshold=200 # {}

IDrecommendations= pd.DataFrame(np.zeros, index=[], columns=['userId', 'ID','longtail no.', 'MAE', 'RMSE', 'recom', 'pred', 'real', 'time']) 
ADrecommendations= pd.DataFrame(np.zeros, index=[], columns=['userId', 'ID','longtail no.', 'MAE', 'RMSE', 'recom', 'pred', 'real', 'time']) 
method='max longtail' # 'max lontail'   'longtail score'


while (len(IDrecommendations)<u):
  rand = random.randrange(1,len(tdata.index))
  userId= tdata.iloc[rand].name
  user_predicted_ratings= pd.DataFrame(np.zeros, index=[], columns=['index','rated_no','rated_sum','predicted_rating','real_rating']) 
  user_unrated= find_unrated_items(userId)
  print(u-len(IDrecommendations))
  for i in range(T):
    h_table= h_table_num(T_H_table,i,r) 
    bucket= bucket_num(T_bucket,i, index) 
    user_h_table= create_his_h_table(h_table ,userId) 
    user_sim_set= find_sim_set(r, userId, user_h_table, bucket) 
    user_predicted_ratings= define_predicting_ratings(userId, i, user_sim_set, user_unrated, user_predicted_ratings) 
    user_predicted_ratings= apply_thershold_and_sort(w,user_predicted_ratings)     # i_set

  IDlist, ID_pred, ID_real, IDlist_ID, IDlist_LongtailNo,ADlist, AD_pred, AD_real, ADlist_ID , ADlist_LongtailNo =IAcreate_list(k,longtail_threshold,
                                                                                                                                user_predicted_ratings,
                                                                                                                                method)
  print(userId,'ID',IDlist)
  print(userId,'ID',ADlist)
  #calculate RMSE , MAE for user
  if len(ID_pred)!=0:
    ID_MAE,ID_RMSE= Evaluation(ID_pred, ID_real)
  if len(AD_pred)!=0:
    AD_MAE,AD_RMSE= Evaluation(AD_pred, AD_real)
  
  IDrecommendations.loc[userId]=[userId, IDlist_ID,IDlist_LongtailNo, ID_MAE, ID_RMSE, IDlist, ID_pred, ID_real, datetime.datetime.now()]
  ADrecommendations.loc[userId]=[userId, ADlist_ID,ADlist_LongtailNo, AD_MAE, AD_RMSE, ADlist, AD_pred, AD_real, datetime.datetime.now()]
  

# update MoviesData
update_MoviesData(dataset, IDrecommendations)
update_MoviesData(dataset, ADrecommendations) 

1


/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


795 ID [564, 10, 931, 705, 120]
795 ID [81, 756, 931, 831, 564]


# **Evaluation**

In [31]:
# Evaluations=pd.DataFrame(np.zeros, index=[], columns=['conditions','k','u','threshold','average_ID','aggregate_D', 'longtail no.', 'MAE', 'RMSE']) 
# path = '/content/drive/My Drive/Evaluations100k.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   Evaluations.to_csv(f) 

In [32]:
Evaluations = pd.read_csv('/content/drive/MyDrive/Evaluations100k.csv')
Evaluations.drop('Unnamed: 0', axis=1, inplace=True)
Evaluations.tail(10)

,conditions,type,k,u,threshold,average_ID,aggregate_D,longtail no.,MAE,RMSE,time
338,ID,reRanking-score log2,5,20,8,0.698754,84,66,1.255317,1.521715,172
339,AD,reRanking-score log2,5,20,8,0.663699,82,89,1.241477,1.540612,272
340,ID,reRanking-ratings priority,5,20,10,0.741531,93,78,1.149877,1.417432,222
341,AD,reRanking-ratings priority,5,20,10,0.722998,95,122,1.123657,1.321911,498
342,ID,reRanking-recom priority,5,20,10,0.658825,84,62,1.028197,1.228620,192
343,AD,reRanking-recom priority,5,20,10,0.585363,82,107,1.008257,1.171456,425
344,ID,reRanking-score log 1.5,5,50,15,0.703980,202,185,1.124540,1.374171,531
345,AD,reRanking-score log 1.5,5,50,15,0.611787,210,323,1.080092,1.357172,1588
346,ID,reRanking-score log 1.5,5,50,15,0.729449,203,189,1.055905,1.281089,572
347,AD,reRanking-score log 1.5,5,50,15,0.638600,214,372,1.059101,1.297861,1988


In [33]:
if  method =='max longtail' or method== 'longtailscore':
   threshold =longtail_threshold
else:threshold= N

Evaluations.loc[len(Evaluations.index)] = ['ID', method, k, len(IDrecommendations), threshold, IDrecommendations['ID'].mean(),
      aggregate_diversity(IDrecommendations), IDrecommendations['longtail no.'].sum(),
      IDrecommendations['MAE'].mean(), IDrecommendations['RMSE'].mean(),IDtime] 

Evaluations.loc[len(Evaluations.index)] = ['AD',method, k, len(ADrecommendations), threshold, ADrecommendations['ID'].mean(),
      aggregate_diversity(ADrecommendations), ADrecommendations['longtail no.'].sum(),
      ADrecommendations['MAE'].mean(), ADrecommendations['RMSE'].mean(),ADtime] 

path = '/content/drive/My Drive/Evaluations100k.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  Evaluations.to_csv(f) 
Evaluations.tail(15)

,conditions,type,k,u,threshold,average_ID,aggregate_D,longtail no.,MAE,RMSE,time
335,AD,reRanking-score log 1.5,5,20,15,0.610165,76,86,1.006440,1.223554,399
336,ID,reRanking-ratings priority,5,20,15,0.726233,87,80,1.078420,1.315657,275
337,AD,reRanking-ratings priority,5,20,15,0.674068,88,172,1.100610,1.297570,1040
338,ID,reRanking-score log2,5,20,8,0.698754,84,66,1.255317,1.521715,172
339,AD,reRanking-score log2,5,20,8,0.663699,82,89,1.241477,1.540612,272
340,ID,reRanking-ratings priority,5,20,10,0.741531,93,78,1.149877,1.417432,222
341,AD,reRanking-ratings priority,5,20,10,0.722998,95,122,1.123657,1.321911,498
342,ID,reRanking-recom priority,5,20,10,0.658825,84,62,1.028197,1.228620,192
343,AD,reRanking-recom priority,5,20,10,0.585363,82,107,1.008257,1.171456,425
344,ID,reRanking-score log 1.5,5,50,15,0.703980,202,185,1.124540,1.374171,531
